<a href="https://colab.research.google.com/github/Sheila-Otieno/Machine-learning/blob/main/Computer_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
#Distinguish between horses and humans
import urllib.request
import zipfile

url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip"
file_name = "horse-or-human.zip"
training_dir = 'horse-or-human/training/'
urllib.request.urlretrieve(url, file_name)

zip_ref = zipfile.ZipFile(file_name, 'r')
zip_ref.extractall(training_dir)
zip_ref.close()

In [ ]:
#validation data
validation_url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip"
validation_file_name = "validation-horse-or-human.zip"
validation_dir = 'horse-or-human/validation/'
urllib.request.urlretrieve(validation_url, validation_file_name)

zip_ref = zipfile.ZipFile(validation_file_name, 'r')
zip_ref.extractall(validation_dir)
zip_ref.close()

In [ ]:
#image generator
#downsize images
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
  training_dir,
  target_size=(300, 300),
  class_mode='binary'
)

Found 1027 images belonging to 2 classes.


In [ ]:
# All images will be rescaled by 1./255
validation_datagen = ImageDataGenerator(rescale=1/255)

validation_generator = train_datagen.flow_from_directory(
        validation_dir,
        target_size=(300, 300),
        class_mode='binary')


Found 256 images belonging to 2 classes.


In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3,3), activation='relu' , 
              input_shape=(300, 300, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 149, 149, 16)     0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 35, 35, 64)      

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

history = model.fit_generator(
    train_generator,
    epochs=15,
    validation_data=validation_generator)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/15
33/33 [==============================] - 95s 3s/step - loss: 0.6368 - accuracy: 0.6456 - val_loss: 1.9849 - val_accuracy: 0.6211
Epoch 2/15
33/33 [==============================] - 92s 3s/step - loss: 0.2728 - accuracy: 0.8900 - val_loss: 1.0938 - val_accuracy: 0.7891
Epoch 3/15
33/33 [==============================] - 92s 3s/step - loss: 0.1678 - accuracy: 0.9377 - val_loss: 1.3917 - val_accuracy: 0.8359
Epoch 4/15
33/33 [==============================] - 92s 3s/step - loss: 0.0777 - accuracy: 0.9718 - val_loss: 2.8101 - val_accuracy: 0.7891
Epoch 5/15
33/33 [==============================] - 93s 3s/step - loss: 0.0831 - accuracy: 0.9669 - val_loss: 1.6815 - val_accuracy: 0.8477
Epoch 6/15
33/33 [==============================] - 95s 3s/step - loss: 0.0033 - accuracy: 0.9990 - val_loss: 4.0777 - val_accuracy: 0.8008
Epoch 7/15
33/33 [==============================] - 93s 3s/step - loss: 0.0903 - accuracy: 0.9776 - val_loss: 2.6812 - val_accuracy: 0.8555
Epoch 8/15
33/33 [==

In [ ]:
#testing the model
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")
 

Saving child-g8ccd2d532_640.jpg to child-g8ccd2d532_640 (1).jpg
Saving horse-g6402ac653_640.jpg to horse-g6402ac653_640.jpg
Saving horses-ga8a6cfc90_640.jpg to horses-ga8a6cfc90_640.jpg
Saving people-gc0cdc69bf_640.jpg to people-gc0cdc69bf_640.jpg
[1.]
child-g8ccd2d532_640.jpg is a human
[0.]
horse-g6402ac653_640.jpg is a horse
[0.]
horses-ga8a6cfc90_640.jpg is a horse
[1.]
people-gc0cdc69bf_640.jpg is a human
